# MCMC2.5: Dense and Sparse Matrices

This section is not necessarily for Marcov chain Monte Carlo itself, but the use of Sparse matrices is important to speed up quantum Monte Carlo simulations.

## Dense matrix

To use dense matrices, LinearAlgebra provides wrapper to BLAS/LAPACK.

In [1]:
using LinearAlgebra

I recommend Intel MKL instead of OpenBLAS. You can check whether MKL is used by the following command:

In [2]:
BLAS.vendor()

:mkl

1D Array is called Vector.

In [3]:
Array{Int64, 1} == Vector{Int64}

true

2D Array is called Matrix, which I later call dense matrix.

In [4]:
Array{Float64, 2} == Matrix{Float64}

true

Matrix and Array of Array are different.

In [5]:
matrix = [1 2
                 3 4]
array =[[1, 2], [3, 4]]
matrix == array

false

Julia's support on 3-rank tensors is limited, but still we can define and use them.

In [6]:
tensor = zeros(Float64, 2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

Be careful, matrices are stored columnwise.

In [7]:
mat = ones(10000, 10000)
mat[:, 1]
mat[1, :]
@time mat[:, 1];
@time mat[1, :];

  0.000029 seconds (7 allocations: 78.375 KiB)
  0.000284 seconds (7 allocations: 78.375 KiB)


Thus, vertical vectors are more important.

In [8]:
[1.0 2.0; 3.0 4.0] * [5.0; 6.0]

2-element Array{Float64,1}:
 17.0
 39.0

## BLAS/LAPACK

Dense matrices are not memory-efficient, but support BLAS/LAPACK. The most important BLAS operation in quantum Monte Carlo is rank-1 update (or other finite-rank updates).

In [9]:
A = rand(Float64, 1000, 1000)
Ainv = inv(A)

1000×1000 Array{Float64,2}:
  0.0691911   -0.0498119   -0.0200318   …  -0.0303933   0.0105963  
 -0.0161936   -0.0958369   -0.0205014      -0.0922788  -0.065451   
 -0.102391     0.245369     0.0638286       0.189055    0.035168   
  0.138479     0.0810568    0.0743991       0.0888117   0.282094   
 -0.0505351    0.133361    -0.0470887      -0.0516247  -0.0580868  
  0.116394    -0.121279    -0.10785     …  -0.0985322  -0.0131008  
 -0.0433004    0.095995     0.033792        0.078654    0.21168    
  0.117508     0.0370799   -0.00640062     -0.0153427   0.0152453  
 -0.0142514   -0.041264     0.0830336       0.0454021   0.13451    
  0.26838     -0.341683    -0.0353651      -0.0910678   0.000506084
  0.0324091   -0.208788     0.268982    …   0.117348    0.354554   
  0.0998302   -0.153596    -0.0190494      -0.141255   -0.11843    
  0.170124    -0.245297     0.0269175      -0.115945    0.067135   
  ⋮                                     ⋱                          
 -0.0965212    0.569

Let's check the Sherman-Morrison formula!
$$\left( A+\vec{u} \vec{v}^T \right)^{-1} = A^{-1} - \frac{A^{-1} \vec{u} \vec{v}^TA^{-1}}{1 + \vec{v}^T A^{-1} \vec{u}}$$
Of course, the vectors are vertical. I define $B = A+\vec{u} \vec{v}^T$

In [10]:
u = rand(Float64, 1000)
v = rand(Float64, 1000)
B = copy(A)
BLAS.ger!(1.0, u, v, B)

1000×1000 Array{Float64,2}:
 1.0013     0.755043  0.134377  0.326421   …  1.16916   0.240584   1.16068  
 1.05762    0.857885  0.306902  1.34071       0.94949   0.964892   1.41136  
 0.882821   0.358848  0.989006  0.396482      0.840259  0.705319   0.652684 
 0.513364   0.664304  0.427847  0.253148      0.473645  0.0896011  0.509204 
 0.256793   0.180063  0.734105  0.341999      0.913645  1.04007    0.270107 
 0.604196   1.64484   0.452531  1.22534    …  0.90295   0.57115    0.693225 
 0.535188   1.26705   0.794883  0.992575      1.04678   1.08012    0.728796 
 0.796058   1.13695   0.280288  1.08852       1.43757   1.08091    0.982656 
 0.0997065  1.27707   0.148688  1.02813       0.796905  0.685032   0.681129 
 0.394139   0.448353  0.654436  0.563835      1.22279   0.565173   1.14069  
 0.679356   0.626496  0.745679  0.951007   …  1.00349   0.283238   0.771503 
 0.8081     0.311108  0.528824  0.709133      1.19081   0.717392   1.03397  
 0.66027    1.06512   1.10615   0.669822      0.

You may need to copy the matrix first because most BLAS operations are destructive. The rank-1 update is a BLAS-2 function, so it is faster than the BLAS-3 function, matrix inversion. That's why we use the Sherman-Morrison formula to calculate $B^{-1}$ when we already know $A^{-1}$.

In [11]:
Binv = copy(Ainv)
BLAS.ger!(-1.0 / (1.0 + v' * Ainv * u), Ainv * u, (v' * Ainv)', Binv)

1000×1000 Array{Float64,2}:
  0.0960227   0.00935102  -0.0681975   …  -0.0593046   -0.0430727
  0.0215639  -0.0125827   -0.0882801      -0.132963    -0.140974 
 -0.171141    0.0937765    0.187243        0.263135     0.172684 
  0.0823058  -0.0428039    0.175236        0.149339     0.394453 
 -0.0636245   0.1045      -0.0235918      -0.0375207   -0.0319051
  0.151934   -0.0429141   -0.171649    …  -0.136827    -0.084189 
 -0.0395679   0.104225     0.0270917       0.0746321    0.204214 
  0.104964    0.00942087   0.0161171      -0.00182654   0.0403359
 -0.0040496  -0.0187693    0.0647203       0.0344096    0.114104 
  0.318981   -0.230109    -0.126199       -0.145591    -0.100707 
  0.0669208  -0.132691     0.20703     …   0.0801617    0.285523 
  0.135792   -0.0743008   -0.0836054      -0.180005    -0.190362 
  0.224242   -0.125968    -0.0702303      -0.174258    -0.0411129
  ⋮                                    ⋱                         
 -0.242391    0.247731     0.258515        0.429

Let's check the results.

In [12]:
norm(B * Binv - I)

8.370469552552803e-11

Iterating rank-1 updates accumulates some error, so sometimes you have to refresh the updated matrix "from scratch." (Be careful because sometimes it is not really from scratch.)

As for LAPACK, most Julia functions on linear algebra are just wrappers of LAPACK.

In [13]:
eigvals(A)

1000-element Array{Complex{Float64},1}:
   499.78019448699865 + 0.0im                
      5.0857171484726 + 7.7301076568891665im 
      5.0857171484726 - 7.7301076568891665im 
       8.810216374705 + 2.745095402305266im  
       8.810216374705 - 2.745095402305266im  
     8.31511027925789 + 3.881993698781938im  
     8.31511027925789 - 3.881993698781938im  
     9.06091097564062 + 0.0im                
     6.65623621482431 + 6.23676493805189im   
     6.65623621482431 - 6.23676493805189im   
    -7.33914531048321 + 5.600597922599758im  
    -7.33914531048321 - 5.600597922599758im  
    3.029489965399616 + 8.659048111050264im  
                      ⋮                      
  0.17646039832674848 - 1.010775320928246im  
 -0.11352064550242583 + 0.8946832074761875im 
 -0.11352064550242583 - 0.8946832074761875im 
   0.0997697610662784 + 0.0im                
  0.10510317447086305 + 0.7213562245531321im 
  0.10510317447086305 - 0.7213562245531321im 
   0.5784175046400952 + 0.33308244580619

eigvals is just a wrapper for LAPACK.geevx!, so you can directely call LAPACK.geevx! instead if you wish.

## Sparse matrix

If your program is intensively using sparse matrices, you should use python instead because Julia only supports CSC matrix. Julia's native support for sparse matrices is not strong, so I do not recommend to write a code using multiple types of sparse matrices in Julia.

In [14]:
using SparseArrays

Let's solve a tight-binding model on the 2D square lattice in a poor man's way, i.e. in the real space.

In [15]:
const L = 30
iter1D = 1 : L
nnbondx = zip(Iterators.product(iter1D, iter1D), Iterators.product((mod1(i + 1, L) for i in iter1D), iter1D))
nnbondy = zip(Iterators.product(iter1D, iter1D), Iterators.product(iter1D, (mod1(i + 1, L) for i in iter1D)))
collect(nnbondx), collect(nnbondy)

(Tuple{Tuple{Int64,Int64},Tuple{Int64,Int64}}[((1, 1), (2, 1)) ((1, 2), (2, 2)) … ((1, 29), (2, 29)) ((1, 30), (2, 30)); ((2, 1), (3, 1)) ((2, 2), (3, 2)) … ((2, 29), (3, 29)) ((2, 30), (3, 30)); … ; ((29, 1), (30, 1)) ((29, 2), (30, 2)) … ((29, 29), (30, 29)) ((29, 30), (30, 30)); ((30, 1), (1, 1)) ((30, 2), (1, 2)) … ((30, 29), (1, 29)) ((30, 30), (1, 30))], Tuple{Tuple{Int64,Int64},Tuple{Int64,Int64}}[((1, 1), (1, 2)) ((1, 2), (1, 3)) … ((1, 29), (1, 30)) ((1, 30), (1, 1)); ((2, 1), (2, 2)) ((2, 2), (2, 3)) … ((2, 29), (2, 30)) ((2, 30), (2, 1)); … ; ((29, 1), (29, 2)) ((29, 2), (29, 3)) … ((29, 29), (29, 30)) ((29, 30), (29, 1)); ((30, 1), (30, 2)) ((30, 2), (30, 3)) … ((30, 29), (30, 30)) ((30, 30), (30, 1))])

These iterators will generate the 2D square lattice.

In [16]:
xytoz(nn::Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}) = (nn[1][2] - 1) * L + nn[1][1], (nn[2][2] - 1) * L + nn[2][1]
nnx = Base.Generator(xytoz, nnbondx)
nny = Base.Generator(xytoz, nnbondy)

Base.Generator{Base.Iterators.Zip2{Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}},Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}}}},typeof(xytoz)}(xytoz, Base.Iterators.Zip2{Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}},Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}}}}(Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}}((1:30, 1:30)), Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}}}((1:30, Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}(getfield(Main, Symbol("##5#6"))(), 1:30)))))

Base.Generator(f, iter) is the same as (f(x) for x in iter).

In [17]:
N = L ^ 2
H = spzeros(Float64, N, N)
for (i, j) in nnx
    H[i, j] = -1.0
    H[j, i] = -1.0
end
for (i, j) in nny
    H[i, j] = -1.0
    H[j, i] = -1.0
end
H

900×900 SparseMatrixCSC{Float64,Int64} with 3600 stored entries:
  [2  ,   1]  =  -1.0
  [30 ,   1]  =  -1.0
  [31 ,   1]  =  -1.0
  [871,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [3  ,   2]  =  -1.0
  [32 ,   2]  =  -1.0
  [872,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [4  ,   3]  =  -1.0
  [33 ,   3]  =  -1.0
  [873,   3]  =  -1.0
  ⋮
  [28 , 898]  =  -1.0
  [868, 898]  =  -1.0
  [897, 898]  =  -1.0
  [899, 898]  =  -1.0
  [29 , 899]  =  -1.0
  [869, 899]  =  -1.0
  [898, 899]  =  -1.0
  [900, 899]  =  -1.0
  [30 , 900]  =  -1.0
  [870, 900]  =  -1.0
  [871, 900]  =  -1.0
  [899, 900]  =  -1.0

Most of the operations for sparse matrices are similar to the ones for dense matrices. However, sparse arrays are more memory-efficient when the components of the matrix is almost zero. Especially, if the matrix is sparse enough, it significantly reduces the matrix muliplication cost from $O(N^3)$ to $O(N)$.

In [20]:
Hdense = Array(H)
H * H
Hdense * Hdense
@time H * H;
@time Hdense * Hdense;

  0.000173 seconds (18 allocations: 268.500 KiB)
  0.035650 seconds (6 allocations: 6.180 MiB)


eigvals does not support sparse matrices, so the calculation of the whole eigenvalues still costs $O(N^3)$. I will discuss this problem later in MCMC5.0.

In [21]:
eigvals(Hdense)

900-element Array{Float64,1}:
 -4.000000000000004 
 -3.9562952014676074
 -3.956295201467607 
 -3.9562952014676056
 -3.9562952014676043
 -3.9125904029352245
 -3.912590402935224 
 -3.9125904029352214
 -3.912590402935216 
 -3.827090915285207 
 -3.827090915285198 
 -3.8270909152851953
 -3.827090915285193 
  ⋮                 
  3.8270909152851997
  3.8270909152852073
  3.82709091528521  
  3.9125904029352205
  3.9125904029352228
  3.912590402935223 
  3.912590402935234 
  3.9562952014676016
  3.956295201467605 
  3.956295201467612 
  3.9562952014676163
  3.999999999999999 

For the square (or cubic, etc.) lattice, you can directly begin from a dense matrix. Here's a smart implementation.

In [22]:
H4d = zeros(Float64, L, L, L, L)
for ((i, j), (k, l)) in nnbondx
    H4d[i, j, k, l] = -1.0
    H4d[k, l, i, j] = -1.0
end
for ((i, j), (k, l)) in nnbondy
    H4d[i, j, k, l] = -1.0
    H4d[k, l, i, j] = -1.0
end
H2d = reshape(H4d, N, N)
eigvals(H2d)

900-element Array{Float64,1}:
 -4.000000000000004 
 -3.9562952014676074
 -3.956295201467607 
 -3.9562952014676056
 -3.9562952014676043
 -3.9125904029352245
 -3.912590402935224 
 -3.9125904029352214
 -3.912590402935216 
 -3.827090915285207 
 -3.827090915285198 
 -3.8270909152851953
 -3.827090915285193 
  ⋮                 
  3.8270909152851997
  3.8270909152852073
  3.82709091528521  
  3.9125904029352205
  3.9125904029352228
  3.912590402935223 
  3.912590402935234 
  3.9562952014676016
  3.956295201467605 
  3.956295201467612 
  3.9562952014676163
  3.999999999999999 

## Iterative solvers

Iterative solvers, especially conjugate gradient methods are important for hybrid Monte Carlo simulations for lattice gauge theories.

~ under construction ~